In [0]:
#Q1
Q1_NF = spark.sql('''
SELECT DISTINCT part.titleId, 
  part.personId
FROM imdb_dev.silver.Participation as part
JOIN imdb_dev.silver.Category as cat on part.categoryId = cat.categoryId
WHERE cat.category = 'director' 
''')

Q1_star = spark.sql(''' 
SELECT DISTINCT part.titleId, 
  part.personId 
FROM imdb_dev.gold.ParticipationFact as part
JOIN imdb_dev.gold.CategoryDim as cat on part.categoryId = cat.categoryId
WHERE cat.category = 'director'
''')

Q1_obt = spark.sql('''
SELECT DISTINCT obt.titleId, 
  obt.personId 
FROM imdb_dev.gold.OneBigTable as obt
WHERE obt.category = 'director'
''')

print("Q1_NF count:", Q1_NF.count())
print("Q1_star count:", Q1_star.count())
print("Q1_obt count:", Q1_obt.count())
print("Q1_NF ∩ Q1_star ∩ Q1_obt:", (Q1_NF.intersect(Q1_star).intersect(Q1_obt)).count())

Q1_NF count: 7980934
Q1_star count: 7980934
Q1_obt count: 7980934
Q1_NF ∩ Q1_star ∩ Q1_obt: 7980934


In [0]:
#Q2
Q2_NF = spark.sql('''
SELECT DISTINCT tit.primaryTitle, 
  part.personId, 
  part.titleId
FROM imdb_dev.silver.Participation as part
JOIN imdb_dev.silver.Title as tit on part.titleId = tit.titleId
WHERE tit.isAdult = TRUE 
  AND tit.startYear > 2000 
  AND tit.endYear < 2010 
  AND tit.runtimeMinutes > 10
''')

Q2_star = spark.sql(''' 
SELECT DISTINCT tit.primaryTitle, 
  part.personId, part.titleId
FROM imdb_dev.gold.ParticipationFact as part
JOIN imdb_dev.gold.TitleDim as tit on tit.titleId = part.titleId
WHERE tit.isAdult = TRUE 
  AND tit.startYear > 2000 
  AND tit.endYear < 2010 
  AND tit.runtimeMinutes > 10
''')

Q2_obt = spark.sql('''
SELECT DISTINCT obt.primaryTitle, 
  obt.personId, 
  obt.titleId
FROM imdb_dev.gold.OneBigTable as obt
WHERE obt.isAdult = TRUE 
  AND obt.startYear > 2000 
  AND obt.endYear < 2010 
  AND obt.runtimeMinutes > 10
''')

print("Q2_NF count:", Q2_NF.count())
print("Q2_star count:", Q2_star.count())
print("Q2_obt count:", Q2_obt.count())
print("Q2_NF ∩ Q2_star ∩ Q2_obt:", (Q2_NF.intersect(Q2_star).intersect(Q2_obt)).count())

Q2_NF count: 383644
Q2_star count: 383644
Q2_obt count: 383644
Q2_NF ∩ Q2_star ∩ Q2_obt: 383644


In [0]:
#Q3
Q3_NF = spark.sql('''
SELECT DISTINCT part.personId, 
  part.titleId, 
  tit.primaryTitle, 
  tit.startYear
FROM imdb_dev.silver.Participation as part
JOIN imdb_dev.silver.Title as tit on part.titleId = tit.titleId
JOIN imdb_dev.silver.GenreOfTitle as got on part.titleId = got.titleId
JOIN imdb_dev.silver.Genre as gen on gen.genreId = got.genreId
WHERE (gen.genre = 'Comedy') 
  AND tit.startYear > 2012 
  AND tit.runtimeMinutes > 60
''')

Q3_star = spark.sql(''' 
SELECT DISTINCT pf.personId, 
  pf.titleId, 
  td.primaryTitle, 
  td.startYear
FROM imdb_dev.gold.ParticipationFact AS pf
JOIN imdb_dev.gold.TitleDim AS td ON pf.titleId = td.titleId
JOIN imdb_dev.gold.GenreBridge AS gb ON pf.titleId = gb.titleId
JOIN imdb_dev.gold.GenreDim AS gd ON gb.genreId = gd.genreId
WHERE gd.genre = 'Comedy'
  AND td.startYear > 2012
  AND td.runtimeMinutes > 60

''')

Q3_obt = spark.sql('''
SELECT DISTINCT obt.personId, 
  obt.titleId, 
  obt.primaryTitle, 
  obt.startYear
FROM imdb_dev.gold.OneBigTable as obt
WHERE obt.genre = 'Comedy'
  AND obt.startYear > 2012
  AND obt.runtimeMinutes > 60
''')

print("Q3_NF count:", Q3_NF.count())
print("Q3_star count:", Q3_star.count())
print("Q3_obt count:", Q3_obt.count())
print("Q3_NF ∩ Q3_star ∩ Q3_obt:", (Q3_NF.intersect(Q3_star).intersect(Q3_obt)).count())

Q3_NF count: 1849367
Q3_star count: 1849367
Q3_obt count: 1849367
Q3_NF ∩ Q3_star ∩ Q3_obt: 1849367


In [0]:
#Q4
Q4_NF = spark.sql('''
SELECT uq_parts.personId, 
  ROUND(avg(CAST(uq_parts.averageRating AS FLOAT)), 1) as average_rating
FROM 
(
  SELECT DISTINCT part.personId,
    part.titleId,
    rat.averageRating
  FROM imdb_dev.silver.Participation as part
  JOIN imdb_dev.silver.TitleRating as rat on part.titleId = rat.titleId
  WHERE rat.numVotes > 200
) as uq_parts
GROUP BY uq_parts.personId
''')

Q4_star = spark.sql(''' 
SELECT uq_parts.personId, 
  ROUND(avg(CAST(uq_parts.averageRating AS FLOAT)), 1) as average_rating
FROM (
  SELECT DISTINCT pf.personId,
    pf.titleId,
    pf.averageRating
  FROM imdb_dev.gold.ParticipationFact AS pf
  WHERE pf.numVotes > 200
    AND pf.averageRating IS NOT NULL
) as uq_parts
GROUP BY uq_parts.personId
''')

Q4_obt = spark.sql('''
SELECT deduped_obt.personId, 
  ROUND(avg(CAST(deduped_obt.averageRating AS FLOAT)), 1) as average_rating
FROM (
  SELECT DISTINCT obt.personId, 
  obt.titleId, 
  obt.averageRating, 
  obt.numVotes
  FROM imdb_dev.gold.OneBigTable as obt
  WHERE obt.numVotes > 200
    AND obt.averageRating IS NOT NULL
) AS deduped_obt
GROUP BY deduped_obt.personId
''')

print("Q4_NF count:", Q4_NF.count())
print("Q4_star count:", Q4_star.count())
print("Q4_obt count:", Q4_obt.count())
print("Q4_NF ∩ Q4_star ∩ Q4_obt:", (Q4_NF.intersect(Q4_star).intersect(Q4_obt)).count())

Q4_NF count: 797972
Q4_star count: 797972
Q4_obt count: 797972
Q4_NF ∩ Q4_star ∩ Q4_obt: 797972


In [0]:
#Q5
Q5_NF = spark.sql('''
SELECT uq_parts.personId, 
  uq_parts.parentTitleId, 
  ROUND(avg(CAST(uq_parts.averageRating AS FLOAT)), 1) as average_rating,
  ROUND(min(CAST(uq_parts.averageRating AS FLOAT)), 1) as minimum_rating,
  ROUND(max(CAST(uq_parts.averageRating AS FLOAT)), 1) as maximum_rating
FROM (
  SELECT DISTINCT part.personId,
    part.titleId,
    ep.parentTitleId,
    rat.averageRating
  FROM imdb_dev.silver.Participation as part
  JOIN imdb_dev.silver.TitleEpisode as ep on part.titleId = ep.titleId 
  JOIN imdb_dev.silver.TitleRating as rat on part.titleId = rat.titleId
  WHERE rat.numVotes > 100
) as uq_parts
GROUP BY uq_parts.personId, uq_parts.parentTitleId
''')

Q5_star = spark.sql(''' 
SELECT uq_parts.personId,
  uq_parts.parentTitleId,
  ROUND(avg(CAST(uq_parts.averageRating AS FLOAT)), 1) as average_rating,
  ROUND(min(CAST(uq_parts.averageRating AS FLOAT)), 1) as minimum_rating,
  ROUND(max(CAST(uq_parts.averageRating AS FLOAT)), 1) as maximum_rating
FROM (
  SELECT DISTINCT pf.personId,
    pf.titleId,
    pf.parentTitleId,
    pf.averageRating
  FROM imdb_dev.gold.ParticipationFact AS pf
  WHERE pf.numVotes > 100
    AND pf.parentTitleId IS NOT NULL
    AND pf.averageRating IS NOT NULL
) as uq_parts
GROUP BY uq_parts.personId, uq_parts.parentTitleId
''')

Q5_obt = spark.sql('''
SELECT deduped_obt.personId, 
  deduped_obt.parentTitleId,
  ROUND(avg(CAST(deduped_obt.averageRating AS FLOAT)), 1) as average_rating,
  ROUND(min(CAST(deduped_obt.averageRating AS FLOAT)), 1) as minimum_rating,
  ROUND(max(CAST(deduped_obt.averageRating AS FLOAT)), 1) as maximum_rating
FROM (
  SELECT DISTINCT obt.personId, 
    obt.titleId, 
    obt.parentTitleId, 
    obt.averageRating, 
    obt.numVotes
  FROM imdb_dev.gold.OneBigTable as obt
  WHERE obt.numVotes > 100
    AND obt.parentTitleId IS NOT NULL
    AND obt.averageRating IS NOT NULL
) AS deduped_obt
GROUP BY deduped_obt.personId, deduped_obt.parentTitleId
''')

print("Q5_NF count:", Q5_NF.count())
print("Q5_star count:", Q5_star.count())
print("Q5_obt count:", Q5_obt.count())
print("Q5_NF ∩ Q5_star ∩ Q5_obt:", (Q5_NF.intersect(Q5_star).intersect(Q5_obt)).count())

Q5_NF count: 552210
Q5_star count: 552210
Q5_obt count: 552210
Q5_NF ∩ Q5_star ∩ Q5_obt: 552210


In [0]:
#Q6
Q6_NF = spark.sql('''
SELECT uq_parts.personId, 
  ROUND(avg(CAST(uq_parts.averageRating AS FLOAT)), 1) as average_rating,
  ROUND(min(CAST(uq_parts.averageRating AS FLOAT)), 1) as minimum_rating,
  ROUND(max(CAST(uq_parts.averageRating AS FLOAT)), 1) as maximum_rating
FROM (
  SELECT DISTINCT part.personId,
  part.titleId,
  rat.averageRating
  FROM imdb_dev.silver.Participation as part
  JOIN imdb_dev.silver.Title as tit on part.titleId = tit.titleId
  JOIN imdb_dev.silver.TitleRating as rat on tit.titleId = rat.titleId
  JOIN imdb_dev.silver.GenreOfTitle as got on part.titleId = got.titleId
  JOIN imdb_dev.silver.Genre as gen on gen.genreId = got.genreId
  WHERE (gen.genre = "Romance") 
    AND tit.startYear > 2012 
    AND tit.runtimeMinutes BETWEEN 60 AND 150
) as uq_parts
GROUP BY uq_parts.personId
ORDER BY average_rating DESC
''')

Q6_star = spark.sql(''' 
SELECT uq_parts.personId,
  ROUND(avg(CAST(uq_parts.averageRating AS FLOAT)), 1) as average_rating,
  ROUND(min(CAST(uq_parts.averageRating AS FLOAT)), 1) as minimum_rating,
  ROUND(max(CAST(uq_parts.averageRating AS FLOAT)), 1) as maximum_rating
FROM (
  SELECT DISTINCT pf.personId,
  pf.titleId,
  pf.averageRating
  FROM imdb_dev.gold.ParticipationFact AS pf
  JOIN imdb_dev.gold.TitleDim AS td ON pf.titleId = td.titleId
  JOIN imdb_dev.gold.GenreBridge AS gb ON pf.titleId = gb.titleId
  JOIN imdb_dev.gold.GenreDim AS gd ON gb.genreId = gd.genreId
  WHERE gd.genre = 'Romance'
    AND td.startYear > 2012
    AND td.runtimeMinutes BETWEEN 60 AND 150
    AND pf.averageRating IS NOT NULL
) as uq_parts
GROUP BY uq_parts.personId
ORDER BY average_rating DESC
''')

Q6_obt = spark.sql('''
SELECT deduped_obt.personId, 
  ROUND(avg(CAST(deduped_obt.averageRating AS FLOAT)), 1) as average_rating,
  ROUND(min(CAST(deduped_obt.averageRating AS FLOAT)), 1) as minimum_rating,
  ROUND(max(CAST(deduped_obt.averageRating AS FLOAT)), 1) as maximum_rating
FROM (
  SELECT DISTINCT obt.personId, 
    obt.titleId, 
    obt.averageRating
  FROM imdb_dev.gold.OneBigTable as obt
  WHERE obt.genre = 'Romance'
    AND obt.startYear > 2012
    AND obt.runtimeMinutes BETWEEN 60 AND 150
    AND obt.averageRating IS NOT NULL
) AS deduped_obt
GROUP BY deduped_obt.personId
ORDER BY average_rating DESC
''')

print("Q6_NF count:", Q6_NF.count())
print("Q6_star count:", Q6_star.count())
print("Q6_obt count:", Q6_obt.count())
print("Q6_NF ∩ Q6_star ∩ Q6_obt:", (Q6_NF.intersect(Q6_star).intersect(Q6_obt)).count())

Q6_NF count: 140098
Q6_star count: 140098
Q6_obt count: 140098
Q6_NF ∩ Q6_star ∩ Q6_obt: 140098


In [0]:
#Q7
Q7_NF = spark.sql('''
WITH relevant_titles AS (
  SELECT DISTINCT got.titleId
  FROM imdb_dev.silver.GenreOfTitle AS got
  JOIN imdb_dev.silver.Genre AS gen ON gen.genreId = got.genreId
  WHERE gen.genre IN ('Comedy', 'Romance')  
)

SELECT uq_parts.personId, 
  SUM(CAST(uq_parts.averageRating * uq_parts.numVotes AS FLOAT)) / SUM(CAST(uq_parts.numVotes AS FLOAT)) AS weighted_average, 
  COUNT(uq_parts.titleId) AS titles_count
FROM (
  SELECT DISTINCT part.personId,
    part.titleId,
    rat.averageRating,
    rat.numVotes
  FROM imdb_dev.silver.Participation as part
  JOIN relevant_titles as revtit on part.titleId = revtit.titleId
  JOIN imdb_dev.silver.TitleRating as rat on part.titleId = rat.titleId
  JOIN imdb_dev.silver.Title AS tit ON tit.titleId = part.titleId
  WHERE tit.startYear BETWEEN 2000 AND 2020
    AND tit.runtimeMinutes BETWEEN 45 AND 100
) as uq_parts
GROUP BY uq_parts.personId
ORDER BY weighted_average DESC, titles_count DESC
''')

Q7_star = spark.sql(''' 
WITH relevant_titles AS (
  SELECT DISTINCT gb.titleId
  FROM imdb_dev.gold.GenreBridge AS gb
  JOIN imdb_dev.gold.GenreDim AS gd ON gb.genreId = gd.genreId
  WHERE gd.genre IN ('Comedy', 'Romance')
)

SELECT uq_parts.personId,
  SUM(CAST(uq_parts.averageRating * uq_parts.numVotes AS FLOAT)) / SUM(CAST(uq_parts.numVotes AS FLOAT)) AS weighted_average,
  COUNT(uq_parts.titleId) AS titles_count
FROM (
  SELECT DISTINCT pf.titleId,
    pf.personId,
    pf.averageRating,
    pf.numVotes
  FROM imdb_dev.gold.ParticipationFact AS pf
  JOIN relevant_titles AS rt ON pf.titleId = rt.titleId
  JOIN imdb_dev.gold.TitleDim AS td ON pf.titleId = td.titleId
  WHERE td.startYear BETWEEN 2000 AND 2020
    AND td.runtimeMinutes BETWEEN 45 AND 100
    AND pf.averageRating is not NULL
) as uq_parts
GROUP BY uq_parts.personId
ORDER BY weighted_average DESC, titles_count DESC
''')

Q7_obt = spark.sql('''
SELECT 
  deduped_obt.personId, 
  SUM(CAST(deduped_obt.averageRating * deduped_obt.numVotes AS FLOAT)) / SUM(CAST(deduped_obt.numVotes AS FLOAT)) AS weighted_average,
  COUNT(deduped_obt.titleId) AS titles_count
FROM (
  SELECT DISTINCT obt.personId, 
    obt.titleId, 
    obt.averageRating, 
    obt.numVotes
  FROM imdb_dev.gold.OneBigTable as obt
  WHERE obt.genre IN ('Comedy', 'Romance')
    AND obt.startYear BETWEEN 2000 AND 2020
    AND obt.runtimeMinutes BETWEEN 45 AND 100
    AND obt.averageRating is not NULL
) AS deduped_obt
GROUP BY deduped_obt.personId
ORDER BY weighted_average DESC, titles_count DESC
''')

print("Q7_NF count:", Q7_NF.count())
print("Q7_star count:", Q7_star.count())
print("Q7_obt count:", Q7_obt.count())
print("Q7_NF ∩ Q7_star ∩ Q7_obt:", (Q7_NF.intersect(Q7_star).intersect(Q7_obt)).count())

Q7_NF count: 373627
Q7_star count: 373627
Q7_obt count: 373627
Q7_NF ∩ Q7_star ∩ Q7_obt: 373627


In [0]:
#Q8
Q8_NF = spark.sql('''
WITH relevant_titles AS (
  SELECT DISTINCT got.titleId
  FROM imdb_dev.silver.GenreOfTitle AS got
  JOIN imdb_dev.silver.Genre AS gen ON gen.genreId = got.genreId
  WHERE gen.genre IN ('Sci-Fi', 'Fantasy', 'Adventure', 'Action')  
)

SELECT uq_parts.personId, 
  SUM(CAST(uq_parts.averageRating * uq_parts.numVotes AS FLOAT)) / SUM(CAST(uq_parts.numVotes AS FLOAT)) AS weighted_average, 
  COUNT(uq_parts.titleId) AS titles_count
FROM (
  SELECT DISTINCT part.personId,
    part.titleId,
    rat.averageRating,
    rat.numVotes
  FROM imdb_dev.silver.Participation as part
  JOIN relevant_titles as revtit on part.titleId = revtit.titleId
  JOIN imdb_dev.silver.TitleRating as rat on part.titleId = rat.titleId
  JOIN imdb_dev.silver.Title AS tit ON tit.titleId = part.titleId
  JOIN imdb_dev.silver.TitleType AS typ ON tit.titleTypeId = typ.titleTypeId
  WHERE (typ.titleType = 'movie' or typ.titleType = 'tvSeries')
    AND tit.startYear > 1980
    AND tit.endYear < 2000
    AND tit.runtimeMinutes > 300
) as uq_parts
GROUP BY uq_parts.personId
HAVING (SUM(uq_parts.averageRating * uq_parts.numVotes) / SUM(uq_parts.numVotes)) > 5.0
ORDER BY weighted_average DESC, titles_count DESC
''')

Q8_star = spark.sql(''' 
WITH relevant_titles AS (
  SELECT DISTINCT gb.titleId
  FROM imdb_dev.gold.GenreBridge AS gb
  JOIN imdb_dev.gold.GenreDim AS gd ON gb.genreId = gd.genreId
  WHERE gd.genre IN ('Sci-Fi', 'Fantasy', 'Adventure', 'Action')
)

SELECT uq_parts.personId,
  SUM(CAST(uq_parts.averageRating * uq_parts.numVotes AS FLOAT)) / SUM(CAST(uq_parts.numVotes AS FLOAT)) AS weighted_average,
  COUNT(uq_parts.titleId) AS titles_count
FROM (
  SELECT DISTINCT pf.personId,
    pf.titleId,
    pf.averageRating,
    pf.numVotes
    FROM imdb_dev.gold.ParticipationFact AS pf
  JOIN relevant_titles AS rt ON pf.titleId = rt.titleId
  JOIN imdb_dev.gold.TitleDim AS td ON pf.titleId = td.titleId
  WHERE td.titleType IN ('movie', 'tvSeries')
    AND td.startYear > 1980
    AND td.endYear < 2000
    AND td.runtimeMinutes > 300
    AND pf.averageRating is not NULL
) as uq_parts
GROUP BY uq_parts.personId
HAVING (SUM(uq_parts.averageRating * uq_parts.numVotes) / SUM(uq_parts.numVotes)) > 5.0
ORDER BY weighted_average DESC, titles_count DESC
''')

Q8_obt = spark.sql('''
SELECT 
  deduped_obt.personId, 
  SUM(CAST(deduped_obt.averageRating * deduped_obt.numVotes AS FLOAT)) / SUM(CAST(deduped_obt.numVotes AS FLOAT)) AS weighted_average,
  COUNT(deduped_obt.titleId) AS titles_count
FROM (
  SELECT DISTINCT obt.personId, obt.titleId, obt.averageRating, obt.numVotes
  FROM imdb_dev.gold.OneBigTable as obt
  WHERE obt.genre IN ('Sci-Fi', 'Fantasy', 'Adventure', 'Action')
    AND obt.titleType IN ('movie', 'tvSeries')
    AND obt.startYear > 1980
    AND obt.endYear < 2000
    AND obt.runtimeMinutes > 300
    AND obt.averageRating is not NULL
) AS deduped_obt
GROUP BY deduped_obt.personId
HAVING SUM(deduped_obt.averageRating * deduped_obt.numVotes) / SUM(deduped_obt.numVotes) > 5.0
ORDER BY weighted_average DESC, titles_count DESC
''')

print("Q8_NF count:", Q8_NF.count())
print("Q8_star count:", Q8_star.count())
print("Q8_obt count:", Q8_obt.count())
print("Q8_NF ∩ Q8_star ∩ Q8_obt:", (Q8_NF.intersect(Q8_star).intersect(Q8_obt)).count())

Q8_NF count: 2145
Q8_star count: 2145
Q8_obt count: 2145
Q8_NF ∩ Q8_star ∩ Q8_obt: 2145


In [0]:
#Q9
Q9_NF = spark.sql('''
WITH relevant_participations AS (
  SELECT DISTINCT prim.personId, kn.titleId
  FROM imdb_dev.silver.PersonPrimaryProfession AS prim
  JOIN imdb_dev.silver.Profession AS per ON prim.professionId = per.professionId
  JOIN imdb_dev.silver.KnownForParticipation AS kn ON prim.personId = kn.personId
  JOIN imdb_dev.silver.Participation AS part ON part.personId = kn.personId and kn.titleId = part.titleId
  WHERE per.profession IN ('director', 'writer', 'producer')  
)

SELECT uq_parts.personId, 
  ROUND(avg(CAST(uq_parts.averageRating AS FLOAT)), 1) as average_rating,
  COUNT(uq_parts.titleId) AS titles_count
FROM (
  SELECT DISTINCT part.personId,
    part.titleId,
    rat.averageRating
  FROM imdb_dev.silver.Participation as part
  JOIN relevant_participations as revpart on part.titleId = revpart.titleId and part.personId = revpart.personId
  JOIN imdb_dev.silver.TitleRating as rat on part.titleId = rat.titleId
  JOIN imdb_dev.silver.Title AS tit ON tit.titleId = part.titleId
  JOIN imdb_dev.silver.TitleType AS typ ON tit.titleTypeId = typ.titleTypeId
  WHERE typ.titleType = 'short'
    AND tit.startYear > 1900
    AND tit.endYear < 2000
    AND tit.runtimeMinutes > 3
) as uq_parts
GROUP BY uq_parts.personId
HAVING COUNT(uq_parts.titleId) > 1
  AND AVG(uq_parts.averageRating) > 6.0
ORDER BY average_rating DESC, titles_count DESC
''')

Q9_star = spark.sql(''' 
WITH relevant_participations AS (
  SELECT DISTINCT ppb.personId, pf.titleId
  FROM imdb_dev.gold.PrimaryProfessionBridge AS ppb
  JOIN imdb_dev.gold.PrimaryProfessionDim AS ppd ON ppb.professionId = ppd.professionId
  JOIN imdb_dev.gold.ParticipationFact AS pf ON pf.personId = ppb.personId
  WHERE ppd.profession IN ('director', 'writer', 'producer')  
  AND pf.isKnownForParticipation = true
)

SELECT uq_parts.personId,
  ROUND(avg(CAST(uq_parts.averageRating AS FLOAT)), 1) as average_rating,
  COUNT(uq_parts.titleId) AS titles_count
FROM (
  SELECT DISTINCT pf.personId,
    pf.titleId,
    pf.averageRating
  FROM imdb_dev.gold.ParticipationFact AS pf
  JOIN relevant_participations AS rp ON pf.personId = rp.personId AND pf.titleId = rp.titleId
  JOIN imdb_dev.gold.TitleDim AS td ON pf.titleId = td.titleId
  WHERE td.titleType = 'short'
    AND td.startYear > 1900
    AND td.endYear < 2000
    AND td.runtimeMinutes > 3
    AND pf.averageRating is not NULL
) as uq_parts
GROUP BY uq_parts.personId
HAVING COUNT(uq_parts.titleId) > 1
  AND AVG(uq_parts.averageRating) > 6.0
ORDER BY average_rating DESC, titles_count DESC
''')

Q9_obt = spark.sql('''
SELECT 
  deduped_obt.personId, 
  ROUND(avg(CAST(deduped_obt.averageRating AS FLOAT)), 1) as average_rating,
  COUNT(deduped_obt.titleId) AS titles_count
FROM (
  SELECT DISTINCT obt.personId, obt.titleId, obt.averageRating
  FROM imdb_dev.gold.OneBigTable as obt
  WHERE obt.profession IN ('director', 'writer', 'producer')
    AND obt.isKnownForParticipation = TRUE
    AND obt.titleType = 'short'
    AND obt.startYear > 1900
    AND obt.endYear < 2000
    AND obt.runtimeMinutes > 3
    AND obt.averageRating is not NULL
) AS deduped_obt
GROUP BY deduped_obt.personId
HAVING COUNT(deduped_obt.titleId) > 1
   AND AVG(deduped_obt.averageRating) > 6.0
ORDER BY average_rating DESC, titles_count DESC
''')

print("Q9_NF count:", Q9_NF.count())
print("Q9_star count:", Q9_star.count())
print("Q9_obt count:", Q9_obt.count())
print("Q9_NF ∩ Q9_star ∩ Q9_obt:", (Q9_NF.intersect(Q9_star).intersect(Q9_obt)).count())

Q9_NF count: 1497
Q9_star count: 1497
Q9_obt count: 1497
Q9_NF ∩ Q9_star ∩ Q9_obt: 1497


In [0]:
#Q10
Q10_NF = spark.sql('''
WITH relevant_participations AS (
  SELECT DISTINCT prim.personId, kn.titleId
  FROM imdb_dev.silver.PersonPrimaryProfession AS prim
  JOIN imdb_dev.silver.KnownForParticipation AS kn ON prim.personId = kn.personId
  JOIN imdb_dev.silver.Profession AS per ON prim.professionId = per.professionId
  JOIN imdb_dev.silver.Participation AS part ON part.personId = kn.personId and kn.titleId = part.titleId
  WHERE per.profession IN ('actor', 'producer')  
),
relevant_titles AS (
  SELECT DISTINCT akas.titleId
  FROM imdb_dev.silver.TitleAKAS AS akas
  JOIN imdb_dev.silver.Region AS reg ON akas.regionId = reg.regionId
  WHERE reg.region IN ('US', 'GB')
)

SELECT uq_parts.titleId, 
  COUNT(uq_parts.personId) AS num_actors_producers
FROM (
  SELECT DISTINCT part.personId,
    part.titleId
  FROM imdb_dev.silver.Participation as part
  JOIN relevant_participations as revpart on part.titleId = revpart.titleId and part.personId = revpart.personId
  JOIN relevant_titles as revtit on part.titleId = revtit.titleId
  JOIN imdb_dev.silver.TitleRating as rat on part.titleId = rat.titleId
  JOIN imdb_dev.silver.Title AS tit ON tit.titleId = part.titleId
  JOIN imdb_dev.silver.TitleType AS typ ON tit.titleTypeId = typ.titleTypeId
  JOIN imdb_dev.silver.Category as cat on part.categoryId = cat.categoryId
  WHERE (typ.titleType = 'short' or typ.titleType = 'tvSeries')
    AND (cat.category = 'actor' or cat.category = 'producer')
    AND rat.averageRating > 3.0
    AND rat.numVotes > 100
) as uq_parts
GROUP BY uq_parts.titleId
HAVING COUNT(uq_parts.personId) BETWEEN 3 and 10
ORDER BY num_actors_producers DESC
''')

Q10_star = spark.sql(''' 
WITH relevant_participations AS (
  SELECT DISTINCT ppb.personId, pf.titleId
  FROM imdb_dev.gold.PrimaryProfessionBridge AS ppb
  JOIN imdb_dev.gold.PrimaryProfessionDim AS ppd ON ppb.professionId = ppd.professionId
  JOIN imdb_dev.gold.ParticipationFact AS pf ON ppb.personId = pf.personId
  WHERE ppd.profession IN ('actor', 'producer')
    AND pf.isKnownForParticipation = TRUE
),
relevant_titles AS (
  SELECT DISTINCT ab.titleId
  FROM imdb_dev.gold.AKASBridge AS ab
  JOIN imdb_dev.gold.AKASDim AS ad ON ab.AKASId = ad.AKASId
  WHERE ad.region IN ('US', 'GB')
)

SELECT uq_parts.titleId,
  COUNT(uq_parts.personId) AS num_actors_producers
FROM (
  SELECT DISTINCT pf.personId,
    pf.titleId
  FROM imdb_dev.gold.ParticipationFact AS pf
  JOIN relevant_participations AS rp ON pf.titleId = rp.titleId AND pf.personId = rp.personId
  JOIN relevant_titles AS rt ON pf.titleId = rt.titleId
  JOIN imdb_dev.gold.TitleDim AS td ON pf.titleId = td.titleId
  JOIN imdb_dev.gold.CategoryDim AS cd ON pf.categoryId = cd.categoryId
  WHERE td.titleType IN ('short', 'tvSeries')
    AND cd.category IN ('actor', 'producer')
    AND pf.averageRating > 3.0
    AND pf.numVotes > 100
) as uq_parts
GROUP BY uq_parts.titleId
HAVING COUNT(uq_parts.personId) BETWEEN 3 AND 10
ORDER BY num_actors_producers DESC
''')

Q10_obt = spark.sql('''
SELECT 
  deduped_obt.titleId, 
  COUNT(deduped_obt.personId) AS num_actors_producers
FROM (
  SELECT DISTINCT titleId, 
    personId
  FROM imdb_dev.gold.OneBigTable as obt
  WHERE obt.profession IN ('actor', 'producer')
    AND obt.isKnownForParticipation = TRUE
    AND obt.region IN ('US', 'GB')
    AND obt.titleType IN ('short', 'tvSeries')
    AND obt.category IN ('actor', 'producer')
    AND obt.averageRating > 3.0
    AND obt.numVotes > 100
) AS deduped_obt
GROUP BY deduped_obt.titleId
HAVING COUNT(deduped_obt.personId) BETWEEN 3 AND 10
ORDER BY num_actors_producers DESC
''')

print("Q10_NF count:", Q10_NF.count())
print("Q10_star count:", Q10_star.count())
print("Q10_obt count:", Q10_obt.count())
print("Q10_NF ∩ Q10_star ∩ Q10_obt:", (Q10_NF.intersect(Q10_star).intersect(Q10_obt)).count())

Q10_NF count: 6798
Q10_star count: 6798
Q10_obt count: 6798
Q10_NF ∩ Q10_star ∩ Q10_obt: 6798
